In [1]:
import numpy as np
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
# from keras.layers.core import Dense, Activation
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from keras.utils import np_utils
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [36]:
iris = datasets.load_iris()
x = preprocessing.scale(iris.data)
y = np_utils.to_categorical(iris.target)
# x_train, x_test, y_train, y_test = train_test_split(x, y, train_size  = 0.7)
# num_classes = y_test.shape[1]

x = [[0,0],[1,0],[0,1],[1,1]]
x_train = [[0,0],[1,0],[0,1],[1,1]]
y_train = [0,0,0,1]
print(x_train)
num_classes = 2

[[0, 0], [1, 0], [0, 1], [1, 1]]


In [37]:
def iris_model(activation="relu", optimizer="adam", out_dim=100):
#     model = Sequential()
#     model.add(Dense(out_dim, input_dim=4, activation=activation))
#     model.add(Dense(out_dim, activation=activation))
#     model.add(Dense(num_classes, activation="softmax"))
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    i = Input(4)
    x = Dense(out_dim, activation=activation)(i)
    x = Dense(out_dim, activation=activation)(x)
    x = Dense(num_classes, activation="softmax")(x)
    model = Model(i, x)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    print("actiation=", activation)
    return model

In [38]:
# activation = ["relu", "sigmoid"]
# optimizer = ["adam", "adagrad"]
# out_dim = [100, 200]
# nb_epoch = [10, 25]
# batch_size = [5, 10]
activation = ["relu"]
nb_epoch = [3]
batch_size = [5]

In [39]:
model = KerasClassifier(build_fn=iris_model, verbose=1)
# param_grid = dict(activation=activation, 
#                   optimizer=optimizer, 
#                   out_dim=out_dim, 
#                   nb_epoch=nb_epoch, 
#                   batch_size=batch_size)
param_grid = dict(activation=activation, 
                  nb_epoch=nb_epoch, 
                  batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2)

In [40]:
grid_result = grid.fit(x_train, y_train)

actiation= relu
actiation= relu
actiation= relu


/Users/dowon/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Error when checking input: expected input_67 to have shape (4,) but got array with shape (1,)

  FitFailedWarning)
/Users/dowon/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Error when checking input: expected input_68 to have shape (4,) but got array with shape (1,)

  FitFailedWarning)


ValueError: Error when checking input: expected input_69 to have shape (4,) but got array with shape (1,)

In [18]:
print (grid_result.best_score_)
print (grid_result.best_params_)

0.8096879422664642
{'activation': 'relu', 'batch_size': 5, 'nb_epoch': 3, 'optimizer': 'adam', 'out_dim': 100}


In [8]:
grid_eval = grid.predict(x_test)
def y_binary(i):
    if   i == 0: return [1, 0, 0]
    elif i == 1: return [0, 1, 0]
    elif i == 2: return [0, 0, 1]
y_eval = np.array([y_binary(i) for i in grid_eval])
accuracy = (y_eval == y_test)
print (np.count_nonzero(accuracy == True) / (accuracy.shape[0] * accuracy.shape[1]))

0.9555555555555556


In [9]:
model = iris_model(activation=grid_result.best_params_['activation'], 
                   optimizer=grid_result.best_params_['optimizer'], 
                   out_dim=grid_result.best_params_['out_dim'])
model.summary()

Model: "model_161"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_162 (InputLayer)       [(None, 4)]               0         
_________________________________________________________________
dense_483 (Dense)            (None, 100)               500       
_________________________________________________________________
dense_484 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_485 (Dense)            (None, 3)                 303       
Total params: 10,903
Trainable params: 10,903
Non-trainable params: 0
_________________________________________________________________
